In [55]:
import pdb
import cPickle as pickle
import numpy as np
import pandas as pd
import os
import math
import matplotlib.pyplot as plt
import pylab as plt
from astropy.wcs import WCS
from astropy.io import fits
import readcol
import parameters
from utils import clean_args
from lmfit import Parameters, minimize, fit_report

%matplotlib inline  

In [57]:
def build_flux_keywords(simstack_output_object):
    z_nodes = simstack_output_object[0]['z_nodes']
    m_nodes = simstack_output_object[0]['m_nodes']
    for i in range(len(z_nodes)):
        for j in range(len(m_nodes)):
            x = ("z_%.1f_%.1f__m_%.1f_%.1f_sf") % (z_nodes[i],z_nodes[i+1],m_nodes[j],m_nodes[j+1]) 
            print clean_args(x)
    kywrd = []
    return kywrd

In [23]:
def load_stacked_fluxes(params, out_file_path, out_file_suffix):
    fpath = "%s/%s_%s_%s.p" % (out_file_path, params['io']['flux_densities_filename'],params['io']['shortname'],out_file_suffix)
    print 'loading '+fpath

    restored_fluxes = pickle.load( open( fpath, "rb" ) )
    
    return restored_fluxes

In [2]:
output_folder           = '../examples/output/' # Output folder
output_bootstrap_folder = '../examples/output/bootstrap_fluxes/' # Output folder for bootstraps
out_file_path   = output_folder
out_file_suffix = ''

In [3]:
parameter_file = '../examples/example.cfg'
params = parameters.get_params(parameter_file)

In [44]:
obs = load_stacked_fluxes(params, out_file_path, out_file_suffix)
nodes = obs[0]
stacked_fluxes = obs[1]
wavelengths =[float(wv) for wv in stacked_fluxes.keys()]
m_nodes = nodes['m_nodes']
print m_nodes
z_nodes = nodes['z_nodes']
print z_nodes

loading ../examples/output//simstack_flux_densities_uVista_Laigle_3color_tests_.p


In [41]:
stacked_fluxes['160.0']['z_0p5_1p0__m_10p0_10p5_sf'].value

0.0028322904601394421

In [58]:
build_flux_keywords(obs)

ERROR: IndexError: list index out of range [IPython.core.interactiveshell]
ERROR:astropy:IndexError: list index out of range


z_0p5_1p0__m_9p0_9p5_sf
z_0p5_1p0__m_9p5_10p0_sf
z_0p5_1p0__m_10p0_10p5_sf
z_0p5_1p0__m_10p5_11p0_sf
z_0p5_1p0__m_11p0_12p0_sf


IndexError: list index out of range